In [1]:
import sys
import math
import csv
import random
import json 
import time

try:
    jparams = json.load(open('params.json'))
except:
    print("ERROR: something is wrong with the params.json file.")
    sys.exit()
    
#-- store the input 3D points in list
list_pts_3d = []
with open(jparams['input-file']) as csvfile:
    r = csv.reader(csvfile, delimiter=' ')
    header = next(r)
    for line in r:
        p = list(map(float, line)) #-- convert each str to a float
        assert(len(p) == 3)
        list_pts_3d.append(p)

In [2]:
import math
import numpy
import scipy.spatial
import startin

In [3]:
class BoundingBox:
    """A 2D bounding box"""
    
    def __init__(self, points):
        if len(points) == 0:
            raise ValueError("Can't compute bounding box of empty list")
        
        self.minx, self.miny = float("inf"), float("inf")
        self.maxx, self.maxy = float("-inf"), float("-inf")
        for x, y, *_ in points:
            # Set min coords
            if x < self.minx:
                self.minx = x
            if y < self.miny:
                self.miny = y
            # Set max coords
            if x > self.maxx:
                self.maxx = x
            elif y > self.maxy:
                self.maxy = y
                
    @property
    def width(self):
        return self.maxx - self.minx
    @property
    def height(self):
        return self.maxy - self.miny
    def __repr__(self):
        return "BoundingBox(X: {} - {}, Y: {} - {})".format(
            self.minx, self.maxx, self.miny, self.maxy)
      
bbox = BoundingBox(list_pts_3d)
bbox

BoundingBox(X: 2.0 - 252.0, Y: 3.0 - 253.0)

In [4]:
class Raster:
  """Simple raster based on ESRI ASCII schema"""
  
  def __init__(self, bbox, cell_size, no_data=-9999):
    self.ncols = math.ceil(bbox.width / cell_size)
    self.nrows = math.ceil(bbox.height / cell_size)
    self.xllcorner = bbox.minx
    self.yllcorner = bbox.miny
    self.cell_size = cell_size
    self.no_data = no_data
    
    # initialize list of values with no_data
    self.values = [self.no_data] * self.ncols * self.nrows
    
  @property
  def centers(self):
    """Cell center coordinates"""
    xulcenter = self.xllcorner + self.cell_size/2
    yulcenter = self.yllcorner + self.cell_size/2 + self.cell_size*self.nrows
    for i in range(self.ncols):
      for j in range(self.nrows):
        x = xulcenter + j * self.cell_size
        y = yulcenter - i * self.cell_size
        yield (x,y)
    
  def to_ascii(self):
    rows = [
      "NCOLS %d" % self.ncols,
      "NROWS %d" % self.nrows,
      "XLLCORNER %s" % self.xllcorner,
      "YLLCORNER %s" % self.yllcorner,
      "CELLSIZE %s" % self.cell_size,
      "NODATA_VALUE %s" % self.no_data,
      ' '.join( (str(x) for x in self.values) )
    ]
    return '\n'.join(rows)

In [5]:
bbox = BoundingBox(list_pts_3d)
raster = Raster(bbox, 30)
type(raster.to_ascii())

str

In [24]:
test
dt.get_point(test[0])

[224.0, 49.0, 120.93194]

In [35]:
dt = startin.DT()
dt.insert(list_pts_3d)

raster.values = []
for center in raster.centers:
    triangle = dt.locate(*center)
    # catch outside on convex hull case
    if not triangle:
        raster.values.append(raster.no_data)
        continue
    # interpolate the triangle verts
    v1, v2, v3 = [dt.get_point(i) for i in triangle]
    # using barycentric coordinate weights 
    # https://codeplea.com/triangular-interpolation
    w1top = (v2[1]-v3[1])*(center[0]-v3[0]) + (v3[0]-v2[0])*(center[1]-v3[1])
    w2top = (v3[1]-v1[1])*(center[0]-v3[0]) + (v1[0]-v3[0])*(center[1]-v3[1])
    bot = (v2[1]-v3[1])*(v1[0]-v3[0]) + (v3[0]-v2[0])*(v1[1]-v3[1])
    w1 = w1top / bot
    w2 = w2top / bot
    w3 = 1 - w1 - w2
    result = v1[2]*w1 + v2[2]*w2 + v3[2]*w3
    raster.values.append(result)

In [36]:
raster.values

[-9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 234.05051333333358,
 108.81303644736863,
 -67.61365742857106,
 53.98028090909099,
 256.1984091304346,
 123.35373218269228,
 51.402738306878305,
 38.5051336637931,
 -9999,
 140.95636419354855,
 64.48927324137935,
 56.82500750000008,
 101.88978857142854,
 158.47013072727265,
 68.59994879545455,
 49.69705578111588,
 49.44648775206612,
 -9999,
 100.47741726495724,
 76.51963060109279,
 129.00878749999998,
 148.13937060869563,
 99.82268291666665,
 87.11758784090908,
 75.42697868131869,
 8.19886515121948,
 -9999,
 73.95510714285683,
 122.9659205333333,
 137.72554583333334,
 161.6146107692307,
 126.41356882352942,
 92.52137057142855,
 49.54109677142844,
 2.2506266249999953,
 -9999,
 100.9807160625,
 71.9260769354837,
 109.25139025423721,
 149.99685264705886,
 128.86432455357146,
 117.8197317037037,
 62.70931349999978,
 14.164563167155421,
 -9999,
 94.